In [1]:
!pip3 install planetterp


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
#Is course 1 harder than course 2?
#u1, m, X will denote Course 1
#u2, n, Y will denote Course 2
#We would like to test the null hypotheis u1 - u2 = 0
#Against the alternative hypothesis u1 - u2 < 0
import planetterp as pt
from scipy import stats
import seaborn as sns

def test_two_courses(id1, id2, incl_w=True):
    letter_gr = {'A+': 4.0, 'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 'C+': 2.3, 'C': 2.0, 'C-': 1.7, 
             'D+': 1.3, 'D': 1.0, 'D-': 0.7, 'F': 0.0}
    
    if incl_w:
        letter_gr['W'] = 0.0

    c1 = pt.grades(course=id1)
    c2 = pt.grades(course=id2)

    gr1 = {}
    gr2 = {}

    for l in letter_gr.keys():
        gr1[l] = sum([sec[l] for sec in c1])
        gr2[l] = sum([sec[l] for sec in c2])


    m = sum(gr1.values())
    n = sum(gr2.values())

    #Calculating average GPAs
    xbar = sum([letter_gr[l] * gr1[l] for l in letter_gr.keys()]) / m
    ybar = sum([letter_gr[l] * gr2[l] for l in letter_gr.keys()]) / n

    print(f"{id1} Average GPA (m = {m})", xbar)
    print(f"{id2} Average GPA (n = {n})", ybar)

    #Standard deviation calculations
    s1 = (sum([sum(sd) for sd in [[(xbar - letter_gr[l]) ** 2 for i in range(gr1[l])] for l in letter_gr.keys()]]) / (m - 1)) ** 0.5
    s2 = (sum([sum(sd) for sd in [[(ybar - letter_gr[l]) ** 2 for i in range(gr2[l])] for l in letter_gr.keys()]]) / (n - 1)) ** 0.5

    se1 = s1 / (m**0.5)
    se2 = s2 / (n**0.5)
    
    v = ((se1**2 + se2**2)**2) / ((se1**4 / (m - 1)) + (se2**4 / (n - 1)))
    
    tval = (xbar - ybar) / (se1**2 + se2**2)**0.5
    print(tval)
    p = stats.t.cdf(tval, v)
    print(f"p={p}")
    
test_two_courses('MATH240', 'MATH241', incl_w=False)


MATH240 Average GPA (m = 8148) 2.6840083456062835
MATH241 Average GPA (n = 15924) 2.8313049485054007
-9.772563934447605
p=8.511300595244062e-23


In [22]:
#Perform an ANOVA for a given list of course IDs
def course_anova(courseid_lst, incl_w=True):
    #Format into a dictionary keyed by course_id, storing frequency of grades
    letter_gr = {'A+': 4.0, 'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 'C+': 2.3, 'C': 2.0, 'C-': 1.7, 
             'D+': 1.3, 'D': 1.0, 'D-': 0.7, 'F': 0.0}
    
    if incl_w:
        letter_gr['W'] = 0.0
        
    courses = [pt.grades(c) for c in courseid_lst]
    grds = {}
    
    for i, c in enumerate(courses):
        cid = courseid_lst[i]
        grds[cid] = []
        for l in letter_gr.keys():
            j = sum([sec[l] for sec in c])
            grds[cid] += [letter_gr[l]] * j
            
    data = list(grds.values())
    
    I = len(data)
    grand_sum = sum([sum(crs_grades) for crs_grades in data])
    n = sum([len(crs) for crs in data])
    
    v1 = I - 1
    v2 = n - I
    
    coeff = (grand_sum**2) / n
    
    sst = sum([sum([g ** 2 for g in crs]) for crs in data]) - coeff
    sstr = sum([(sum(crs) ** 2) / len(crs) for crs in data]) - coeff
    sse = sst - sstr
    
    mstr = sstr / v1
    mse = sse / v2
    
    fval = mstr / mse
    p = stats.f.sf(fval, v1, v2)
    print(fval, p)
    print(v1, v2)
    
    
course_anova(['CMSC131', 'CMSC132'], incl_w=False)

2.9882245330057597 0.08388545094645997
1 23086
